In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
from torch.utils.data import Dataset, DataLoader
class DataGenerator(Dataset):
	def __init__(self, path):
		self.files = self.get_files(path)
	def __len__(self):
		return len(self.files)
	def __getitem__(self,idx):
		return data, cap

def load_data(data_path, batch_size=1, num_workers=10, shuffle=True):

	dataset = DataGenerator(data_path)
	data_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)

	return data_loader

In [6]:
import random
from torch.utils.data import Dataset, DataLoader
import cv2
import os
import re
import torch
import numpy as np

use_cuda = torch.cuda.is_available()
print('use_cuda: {}'.format(use_cuda))
device = torch.device("cuda" if use_cuda else "cpu")

class DataGenerator(Dataset):
    def __init__(self, data, word2int):
        self.files = data
        self.word2int = word2int

    def __len__(self):
        return len(self.files)
    def __getitem__(self,idx):
        img_path, cap = self.files[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        return img, cap

    def collate_fn_customised(self, data):

        img_list = []
        cap_list = []
        for i, c in data:
            img_list.append(i)
            cap_list.append(c)

        ##OPERATING ON CAPTION
        modified_cap = []
        for cap in cap_list:
            #Removing Functuatoons and Numbers and converting it into lower case and breaking it into tokens.
            cap = re.sub('[^a-zA-Z ]+', '', cap.lower()).split()
            #Added BOS and EOS to the start and the end of the sent. respectively.
            cap = ["<BOS>"] + cap + ["<EOS>"]
            modified_cap.append(cap)
        max_len = max([len(cap) for cap in modified_cap])
        cap_list = []
        for cap in modified_cap:
            while(len(cap)<max_len):
                cap = cap + ["<PAD>"]
            cap_list.append(cap)
        modified_cap = []
        for cap in cap_list:
            numericalizaed_sentences = []
            for word in cap:
                try:
                    numericalizaed_sentences.append(self.word2int[word])
                except:
                    numericalizaed_sentences.append(self.word2int["<UNK>"])
            modified_cap.append(numericalizaed_sentences)

        cap_array = np.array(modified_cap)
        cap_tensor = torch.tensor(cap_array)
        img_array = np.array(img_list)
        img_tensor = torch.tensor(img_array)

        return img_tensor, cap_tensor

def load_data(data, word2int, batch_size=1, num_workers=10, shuffle=True):

	dataset = DataGenerator(data, word2int)
	data_loader = DataLoader(dataset, collate_fn= dataset.collate_fn_customised, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)

	return data_loader

def train_val_test_split(img_folder, annotation_path):

    lines = open(annotation_path, "r").readlines()
    lines = lines[1:]

    #Populated a dictionary in the format of key=img and value = [cap1, cap2, ...]
    image_caption_dict = {}
    for l in lines:
        img = l.split(',')[0]
        cap = l.split(',')[1][:-1]
        if img in image_caption_dict.keys():
            image_caption_dict[img].append(cap)
        else:
            image_caption_dict[img] = [cap]

    images = list(set([l[:-1].split(',')[0] for l in lines]))
    random.seed(10)
    random.shuffle(images)

    #Splitting the list of images into 70, 10, 20 % split in train, val, test respectively.
    train_images = images[:int(0.7*len(images))]
    val_images = images[int(0.7*len(images)):int(0.8*len(images))]
    test_images = images[int(0.8*len(images)):]

    train_list = [(os.path.join(img_folder, img), cap) for img in train_images for cap in image_caption_dict[img] if os.path.exists(os.path.join(img_folder, img))]
    val_list = [(os.path.join(img_folder, img), cap) for img in val_images for cap in image_caption_dict[img] if os.path.exists(os.path.join(img_folder, img))]
    test_list = [(os.path.join(img_folder, img), cap) for img in test_images for cap in image_caption_dict[img] if os.path.exists(os.path.join(img_folder, img))]

    return train_list, val_list, test_list


def train_epoch(train_loader):

     for img_tensor, cap_tensor in train_loader:
        img_tensor = img_tensor.to(device)
        cap_tensor = cap_tensor.to(device)

        print(img_tensor.shape, cap_tensor.shape)

def val_epoch():
    pass
def test_epoch():
    pass

def train_val_test(train_loader):

    train_epoch(train_loader)

def main():

    data_folder = "/content/drive/MyDrive/data"
    image_folder = os.path.join(data_folder, "flickr30k-images")
    #data_folder = "/content/drive/MyDrive/projects/flkr_image_capt"
    #image_folder = os.path.join(data_folder, "Images")
    #cap_folder = "/content/drive/MyDrive/projects/flkr_image_capt"
    #annotation_path = os.path.join(cap_folder, "captions.txt")
    annotation_path = os.path.join(data_folder, "captions_30k.txt")
    train_list, val_list, test_list = train_val_test_split(image_folder, annotation_path)

    #Preparing Dictionary
    words = []
    for _, cap in train_list:
        cap = re.sub('[^a-zA-Z ]+', '', cap.lower()).split()
        words.extend(cap)
    words = list(set(words))
    words.sort()
    words = ["<BOS>", "<EOS>", "<PAD>", "<UNK>"] + words
    word2int = {}
    int2word = {}
    for id, word in enumerate(words):
        word2int[word] = id
        int2word[id] = word

    train_loader = load_data(val_list, word2int, batch_size=128, num_workers=1)

    train_val_test(train_loader)

main()


use_cuda: False
torch.Size([128, 224, 224, 3]) torch.Size([128, 56])
torch.Size([128, 224, 224, 3]) torch.Size([128, 33])
torch.Size([128, 224, 224, 3]) torch.Size([128, 30])
torch.Size([128, 224, 224, 3]) torch.Size([128, 44])
torch.Size([128, 224, 224, 3]) torch.Size([128, 35])
torch.Size([128, 224, 224, 3]) torch.Size([128, 27])
torch.Size([128, 224, 224, 3]) torch.Size([128, 24])
torch.Size([128, 224, 224, 3]) torch.Size([128, 31])
torch.Size([128, 224, 224, 3]) torch.Size([128, 38])
torch.Size([128, 224, 224, 3]) torch.Size([128, 36])
torch.Size([128, 224, 224, 3]) torch.Size([128, 33])
torch.Size([128, 224, 224, 3]) torch.Size([128, 48])
torch.Size([128, 224, 224, 3]) torch.Size([128, 39])
torch.Size([128, 224, 224, 3]) torch.Size([128, 39])
torch.Size([128, 224, 224, 3]) torch.Size([128, 27])
torch.Size([128, 224, 224, 3]) torch.Size([128, 36])
torch.Size([128, 224, 224, 3]) torch.Size([128, 34])
torch.Size([128, 224, 224, 3]) torch.Size([128, 41])
torch.Size([128, 224, 224, 3])